#### Restart and Run All

In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 8, 28)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-08-28
yesterday: 2023-08-25


In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1308101,WHART,2023-08-23,2023-08-25,2,10.50,10.40,-0.10,-0.95%,-1,-1,10.60,10.40,"2,364,058",2023-08-27,2023-08-27,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,128,WHART,0.00,0.00,I,0.00,10.60,0.00,0.00,11.70,"10,000",-4,4,"30,000",0.000000,0.000000,COS,SET


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-08-25'
ORDER BY a.name



,name
0,AH
1,AIMIRT
2,ASK
3,ASP
4,AWC
5,BBL
6,BCH
7,BEM
8,BH
9,CK


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-08-25' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AH,2023-08-24,2023-08-25,34.25,35.00,"2,767,871",35.00,34.00,B,SET,0.75,2.19%
1,AWC,2023-08-25,2023-08-25,4.10,4.10,"48,252,126",4.14,4.06,B,SET50,0.00,0.00%
2,DIF,2023-08-25,2023-08-25,9.30,9.30,"17,514,189",9.30,9.15,B,SET,0.00,0.00%
3,JASIF,2023-08-24,2023-08-25,6.85,6.95,"24,581,311",7.00,6.85,B,SET,0.10,1.46%
4,JMART,2023-08-25,2023-08-25,22.30,22.30,"43,499,248",22.60,21.20,B,SET50,0.00,0.00%
5,JMT,2023-08-25,2023-08-25,46.50,46.50,"12,636,443",46.75,45.50,B,SET50,0.00,0.00%
6,MCS,2023-08-25,2023-08-25,6.15,6.15,"260,648",6.20,6.00,B,SET,0.00,0.00%
7,NER,2023-08-24,2023-08-25,4.62,4.64,"20,252,589",4.64,4.56,B,SET,0.02,0.43%
8,ORI,2023-08-22,2023-08-25,10.30,10.80,"15,016,281",10.80,10.10,B,SET100,0.50,4.85%
9,PTTGC,2023-08-24,2023-08-25,36.50,38.00,"37,053,264",38.00,35.75,B,SET50,1.50,4.11%


In [11]:
df.shape

(48, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AH,2023-08-24,2023-08-25,34.25,35.00,"2,767,871",35.00,34.00,2.19%,B
1,AWC,2023-08-25,2023-08-25,4.10,4.10,"48,252,126",4.14,4.06,0.00%,B
2,DIF,2023-08-25,2023-08-25,9.30,9.30,"17,514,189",9.30,9.15,0.00%,B
3,JASIF,2023-08-24,2023-08-25,6.85,6.95,"24,581,311",7.00,6.85,1.46%,B
4,JMART,2023-08-25,2023-08-25,22.30,22.30,"43,499,248",22.60,21.20,0.00%,B
5,JMT,2023-08-25,2023-08-25,46.50,46.50,"12,636,443",46.75,45.50,0.00%,B
6,MCS,2023-08-25,2023-08-25,6.15,6.15,"260,648",6.20,6.00,0.00%,B
7,NER,2023-08-24,2023-08-25,4.62,4.64,"20,252,589",4.64,4.56,0.43%,B
8,ORI,2023-08-22,2023-08-25,10.30,10.80,"15,016,281",10.80,10.10,4.85%,B
9,PTTGC,2023-08-24,2023-08-25,36.50,38.00,"37,053,264",38.00,35.75,4.11%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(48, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AH,2023-08-24,2023-08-25,34.25,35.00,2767871,35.00,34.00,2.19,B
1,AWC,2023-08-25,2023-08-25,4.10,4.10,48252126,4.14,4.06,0.00,B
2,DIF,2023-08-25,2023-08-25,9.30,9.30,17514189,9.30,9.15,0.00,B
3,JASIF,2023-08-24,2023-08-25,6.85,6.95,24581311,7.00,6.85,1.46,B
4,JMART,2023-08-25,2023-08-25,22.30,22.30,43499248,22.60,21.20,0.00,B
5,JMT,2023-08-25,2023-08-25,46.50,46.50,12636443,46.75,45.50,0.00,B
6,MCS,2023-08-25,2023-08-25,6.15,6.15,260648,6.20,6.00,0.00,B
7,NER,2023-08-24,2023-08-25,4.62,4.64,20252589,4.64,4.56,0.43,B
8,ORI,2023-08-22,2023-08-25,10.30,10.80,15016281,10.80,10.10,4.85,B
9,PTTGC,2023-08-24,2023-08-25,36.50,38.00,37053264,38.00,35.75,4.11,B


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-08-28,1.80,-0.01,-0.552486,1.81,1.82,1.79,5146394,9294.57691
1,ADVANC,2023-08-28,219.00,1.00,0.458716,218.00,219.00,216.00,3480883,757652.06700
2,AEONTS,2023-08-28,175.50,-3.00,-1.680672,178.00,179.00,174.50,244201,42917.37750
3,AH,2023-08-28,34.75,-0.25,-0.714286,35.25,35.50,34.50,2442121,85310.77650
4,AIMIRT,2023-08-28,10.90,-0.10,-0.909091,10.90,11.00,10.90,254105,2785.62500
...,...,...,...,...,...,...,...,...,...,...
185,WHAIR,2023-08-28,6.90,-0.05,-0.719424,6.95,6.95,6.85,291535,2007.94715
186,WHART,2023-08-28,10.50,0.10,0.961538,10.40,10.50,10.40,840067,8764.25670
187,WHAUP,2023-08-28,3.98,0.00,0.000000,4.00,4.02,3.98,1528115,6092.24824
188,WICE,2023-08-28,8.55,0.40,4.907975,8.40,8.60,8.35,2576291,21813.42405


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,AH,2023-08-24,2023-08-25,34.25,35.00,2767871,35.00,34.00,2.19,B,2023-08-28,34.75,-0.25,-0.714286,35.25,35.50,34.50,2442121,8.531078e+04
1,AWC,2023-08-25,2023-08-25,4.10,4.10,48252126,4.14,4.06,0.00,B,2023-08-28,4.16,0.06,1.463415,4.12,4.20,4.10,54464131,2.261102e+05
2,DIF,2023-08-25,2023-08-25,9.30,9.30,17514189,9.30,9.15,0.00,B,2023-08-28,9.35,0.05,0.537634,9.35,9.45,9.25,12533836,1.172368e+05
3,JASIF,2023-08-24,2023-08-25,6.85,6.95,24581311,7.00,6.85,1.46,B,2023-08-28,6.90,-0.05,-0.719424,6.95,6.95,6.85,3990283,2.754541e+04
4,JMART,2023-08-25,2023-08-25,22.30,22.30,43499248,22.60,21.20,0.00,B,2023-08-28,22.40,0.10,0.448430,22.30,22.90,22.20,34444497,7.758265e+05
5,JMT,2023-08-25,2023-08-25,46.50,46.50,12636443,46.75,45.50,0.00,B,2023-08-28,46.75,0.25,0.537634,46.75,47.25,46.25,11130225,5.199264e+05
6,MCS,2023-08-25,2023-08-25,6.15,6.15,260648,6.20,6.00,0.00,B,2023-08-28,6.35,0.20,3.252033,6.20,6.35,6.15,513858,3.228796e+03
7,NER,2023-08-24,2023-08-25,4.62,4.64,20252589,4.64,4.56,0.43,B,2023-08-28,4.68,0.04,0.862069,4.64,4.70,4.60,11340063,5.269500e+04
8,ORI,2023-08-22,2023-08-25,10.30,10.80,15016281,10.80,10.10,4.85,B,2023-08-28,10.70,-0.10,-0.925926,10.70,10.80,10.60,4728252,5.033294e+04
9,PTTGC,2023-08-24,2023-08-25,36.50,38.00,37053264,38.00,35.75,4.11,B,2023-08-28,38.00,0.00,0.000000,38.25,38.25,37.50,15119406,5.735189e+05


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AH,2023-08-24,2023-08-25,34.25,35.00,2767871,35.00,34.00,2.19,B,34.75,-0.25,2442121,2023-08-28
1,AWC,2023-08-25,2023-08-25,4.10,4.10,48252126,4.14,4.06,0.00,B,4.16,0.06,54464131,2023-08-28
2,DIF,2023-08-25,2023-08-25,9.30,9.30,17514189,9.30,9.15,0.00,B,9.35,0.05,12533836,2023-08-28
3,JASIF,2023-08-24,2023-08-25,6.85,6.95,24581311,7.00,6.85,1.46,B,6.90,-0.05,3990283,2023-08-28
4,JMART,2023-08-25,2023-08-25,22.30,22.30,43499248,22.60,21.20,0.00,B,22.40,0.10,34444497,2023-08-28
5,JMT,2023-08-25,2023-08-25,46.50,46.50,12636443,46.75,45.50,0.00,B,46.75,0.25,11130225,2023-08-28
6,MCS,2023-08-25,2023-08-25,6.15,6.15,260648,6.20,6.00,0.00,B,6.35,0.20,513858,2023-08-28
7,NER,2023-08-24,2023-08-25,4.62,4.64,20252589,4.64,4.56,0.43,B,4.68,0.04,11340063,2023-08-28
8,ORI,2023-08-22,2023-08-25,10.30,10.80,15016281,10.80,10.10,4.85,B,10.70,-0.10,4728252,2023-08-28
9,PTTGC,2023-08-24,2023-08-25,36.50,38.00,37053264,38.00,35.75,4.11,B,38.00,0.00,15119406,2023-08-28


In [20]:
df_merge[colu].shape

(48, 14)

In [21]:
df_merge.shape

(48, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-08-28 2023-08-28  35.00  34.75  -0.71%   2,767,871  35.00  34.00 B Insert
AIMIRT   2023-08-25 2023-08-28  11.00  10.90  -0.91%     151,902  11.00  10.80 I Update
ASK      2023-08-28 2023-08-28  25.75  25.25  -1.94%   3,193,293  25.75  23.70 I Insert
ASP      2023-08-28 2023-08-28   3.00   2.92  -2.67%  22,712,390   3.02   2.80 I Insert
AWC      2023-08-25 2023-08-28   4.10   4.16   1.46%  48,252,126   4.16   4.06 B Update
BBL      2023-08-25 2023-08-28 167.50 169.50   1.19%   4,574,076 169.50 166.50 O Update
BCH      2023-08-22 2023-08-28  18.80  19.50   3.72%  45,000,777  19.50  18.60 I Update
BEM      2023-08-25 2023-08-28   9.05   8.70  -3.87%  15,269,956   9.10   8.70 O Update
BH       2023-08-25 2023-08-28 258.00 255.00  -1.16%   1,666,045 260.00 255.00 O Update
CK       2023-08-28 2023-08-28  

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb